In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp as OR

In [2]:
foods = pd.read_csv('diet_neos/neos_foods.csv', index_col=0)
nutrients = pd.read_csv('diet_neos/neos_nutrients.csv', index_col=0)

In [3]:
def diet(foods, nutrients, integer=False):
    """A model for solving the diet problem.
    
    Args:
        foods (pd.DataFrame): Foods with cost per serving, min and max servings, and nutrients per serving.
        nutrients (pd.DataFrame): Nutrients with min and max bounds.
    """
    FOODS = list(foods.index)                                 # foods
    NUTRIENTS = list(nutrients.index)                         # nutrients
    c = foods['Cost'].to_dict()                               # cost per serving of food 
    f_min = foods['Min'].to_dict()                            # lower bound of food serving
    f_max = foods['Max'].to_dict()                            # upper bound of food serving
    n_min = nutrients['Min'].to_dict()                        # lower bound of nutrient
    n_max = nutrients['Max'].to_dict()                        # upper bound of nutrient  
    a = foods[list(nutrients.index)].transpose().to_dict()    # amt of nutrients per serving of food
    
    # define model
    if integer:
        m = OR.Solver('diet', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    else:
        m = OR.Solver('diet', OR.Solver.CLP_LINEAR_PROGRAMMING)
        
    # decision variables
    x = {}    
    for i in FOODS:
        if integer:
            x[i] = m.IntVar(0, m.infinity(), 'x_%s' % (i)) 
        else:
            x[i] = m.NumVar(0, m.infinity(), 'x_%s' % (i)) 
        
    # objective function.
    m.Minimize(sum(c[i]*x[i] for i in FOODS))
    
    # enforce lower and upper bound on food servings
    for i in FOODS:
        m.Add(x[i] >= f_min[i], name='lb_%s' % (i))
        m.Add(x[i] <= f_max[i], name='ub_%s' % (i))
    
    # enforce lower and upper bound on nutrients 
    for j in NUTRIENTS:
        m.Add(sum(a[i][j]*x[i] for i in FOODS) >= n_min[j], name='lb_%s' % (j))
        m.Add(sum(a[i][j]*x[i] for i in FOODS) <= n_max[j], name='ub_%s' % (j))
        
    return m, x

In [4]:
def solve(m):
    m.Solve()
    print('Solution:')
    print('Objective value =', m.Objective().Value())
    for var in m.variables():
        print(var.name(), ':',  var.solution_value())

In [5]:
m, x = diet(foods, nutrients)
solve(m)

Solution:
Objective value = 0.9560078273216122
x_Frozen Broccoli : 0.0
x_Carrots, Raw : 0.23581781088978732
x_Celery, Raw : 0.0
x_Frozen Corn : 0.0
x_Lettuce, Iceberg,Raw : 0.0
x_Peppers, Sweet, Raw : 0.0
x_Potatoes, Baked : 3.5449447765210538
x_Tofu : 0.0
x_Roasted Chicken : 0.0
x_Spaghetti W/ Sauce : 0.0
x_Tomato,Red,Ripe,Raw : 0.0
x_Apple, Raw, w/Skin : 0.0
x_Banana : 0.0
x_Grapes : 0.0
x_Kiwifruit, Raw, Fresh : 0.0
x_Oranges : 0.0
x_Bagels : 0.0
x_Wheat Bread : 0.0
x_White Bread : 0.0
x_Oatmeal Cookies : 0.0
x_Apple Pie : 0.0
x_Chocolate Chip Cookies : 0.0
x_Butter, Regular : 0.0
x_Cheddar Cheese : 0.0
x_3.3% Fat, Whole Milk : 0.0
x_2% Lowfat Milk : 0.0
x_Skim Milk : 2.167849362594859
x_Poached Eggs : 0.0
x_Scrambled Eggs : 0.0
x_Bologna, Turkey : 0.0
x_Frankfurter, Beef : 0.0
x_Ham, Sliced, Extralean : 0.0
x_Kielbasa, Pork : 0.0
x_Cap'N Crunch : 0.0
x_Cheerios : 0.0
x_Corn Flakes, Kellogg'S : 0.0
x_Raisin Bran, Kellogg'S : 0.0
x_Rice Krispies : 0.0
x_Special K : 0.0
x_Oatmeal : 0.

In [6]:
m, x = diet(foods, nutrients, integer=True)
solve(m)

Solution:
Objective value = 1.1400000000000001
x_Frozen Broccoli : 0.0
x_Carrots, Raw : 1.0
x_Celery, Raw : 0.0
x_Frozen Corn : 0.0
x_Lettuce, Iceberg,Raw : 0.0
x_Peppers, Sweet, Raw : 0.0
x_Potatoes, Baked : 4.0
x_Tofu : 0.0
x_Roasted Chicken : 0.0
x_Spaghetti W/ Sauce : 0.0
x_Tomato,Red,Ripe,Raw : 0.0
x_Apple, Raw, w/Skin : 0.0
x_Banana : 0.0
x_Grapes : 0.0
x_Kiwifruit, Raw, Fresh : 0.0
x_Oranges : 0.0
x_Bagels : 0.0
x_Wheat Bread : 0.0
x_White Bread : 0.0
x_Oatmeal Cookies : 0.0
x_Apple Pie : 0.0
x_Chocolate Chip Cookies : 1.0
x_Butter, Regular : 0.0
x_Cheddar Cheese : 0.0
x_3.3% Fat, Whole Milk : 0.0
x_2% Lowfat Milk : 0.0
x_Skim Milk : 3.0
x_Poached Eggs : 1.0
x_Scrambled Eggs : 0.0
x_Bologna, Turkey : 0.0
x_Frankfurter, Beef : 0.0
x_Ham, Sliced, Extralean : 0.0
x_Kielbasa, Pork : 0.0
x_Cap'N Crunch : 0.0
x_Cheerios : 0.0
x_Corn Flakes, Kellogg'S : 0.0
x_Raisin Bran, Kellogg'S : 0.0
x_Rice Krispies : 0.0
x_Special K : 0.0
x_Oatmeal : 0.0
x_Malt-O-Meal, Choc : 0.0
x_Pizza w/Peppero